# Demo
We'll gather all the functions from the other notebooks and generate questions for an entirely new text.

You can also run this notebook independently with any text and see the results. 

## Imports

In [1]:
# Common imports
import pandas as pd
from IPython.display import Markdown, display, clear_output

### Pickling

In [2]:
import _pickle as cPickle
from pathlib import Path

def dumpPickle(fileName, content):
    pickleFile = open(fileName, 'wb')
    cPickle.dump(content, pickleFile, -1)
    pickleFile.close()

def loadPickle(fileName):    
    file = open(fileName, 'rb')
    content = cPickle.load(file)
    file.close()
    
    return content
    
def pickleExists(fileName):
    file = Path(fileName)
    
    if file.is_file():
        return True
    
    return False

## *Extract all words from plain text and generate it's features*

In [3]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

#Extract answers and the sentence they are in
def extractAnswers(qas, doc):
    answers = []

    senStart = 0
    senId = 0

    for sentence in doc.sents:
        senLen = len(sentence.text)

        for answer in qas:
            answerStart = answer['answers'][0]['answer_start']

            if (answerStart >= senStart and answerStart < (senStart + senLen)):
                answers.append({'sentenceId': senId, 'text': answer['answers'][0]['text']})

        senStart += senLen
        senId += 1
    
    return answers

#TODO - Clean answers from stopwords?
def tokenIsAnswer(token, sentenceId, answers):
    for i in range(len(answers)):
        if (answers[i]['sentenceId'] == sentenceId):
            if (answers[i]['text'] == token):
                return True
    return False

#Save named entities start points

def getNEStartIndexs(doc):
    neStarts = {}
    for ne in doc.ents:
        neStarts[ne.start] = ne
        
    return neStarts 

def getSentenceStartIndexes(doc):
    senStarts = []
    
    for sentence in doc.sents:
        senStarts.append(sentence[0].i)
    
    return senStarts
    
def getSentenceForWordPosition(wordPos, senStarts):
    for i in range(1, len(senStarts)):
        if (wordPos < senStarts[i]):
            return i - 1
        
def addWordsForParagrapgh(newWords, text):
    doc = nlp(text)

    neStarts = getNEStartIndexs(doc)
    senStarts = getSentenceStartIndexes(doc)
    
    #index of word in spacy doc text
    i = 0
    
    while (i < len(doc)):
        #If the token is a start of a Named Entity, add it and push to index to end of the NE
        if (i in neStarts):
            word = neStarts[i]
            #add word
            currentSentence = getSentenceForWordPosition(word.start, senStarts)
            wordLen = word.end - word.start
            shape = ''
            for wordIndex in range(word.start, word.end):
                shape += (' ' + doc[wordIndex].shape_)

            newWords.append([word.text,
                            0,
                            0,
                            currentSentence,
                            wordLen,
                            word.label_,
                            None,
                            None,
                            None,
                            shape])
            i = neStarts[i].end - 1
        #If not a NE, add the word if it's not a stopword or a non-alpha (not regular letters)
        else:
            if (doc[i].is_stop == False and doc[i].is_alpha == True):
                word = doc[i]

                currentSentence = getSentenceForWordPosition(i, senStarts)
                wordLen = 1

                newWords.append([word.text,
                                0,
                                0,
                                currentSentence,
                                wordLen,
                                None,
                                word.pos_,
                                word.tag_,
                                word.dep_,
                                word.shape_])
        i += 1

def oneHotEncodeColumns(df):
    columnsToEncode = ['NER', 'POS', "TAG", 'DEP']

    for column in columnsToEncode:
        one_hot = pd.get_dummies(df[column])
        one_hot = one_hot.add_prefix(column + '_')

        df = df.drop(column, axis = 1)
        df = df.join(one_hot)
    
    return df

## *Predict whether a word is a keyword* 

In [4]:
def generateDf(text):
    words = []
    addWordsForParagrapgh(words, text)

    wordColums = ['text', 'titleId', 'paragrapghId', 'sentenceId','wordCount', 'NER', 'POS', 'TAG', 'DEP','shape']
    df = pd.DataFrame(words, columns=wordColums)
    
    return df

In [5]:
def prepareDf(df):
    #One-hot encoding
    wordsDf = oneHotEncodeColumns(df)


    #Add missing colums 
    predictorFeaturesName = 'data/pickles/nb-predictor-features.pkl'
    featureNames = loadPickle(predictorFeaturesName)

    for feature in featureNames:
        if feature not in wordsDf.columns:
            wordsDf[feature] = 0    
                
    #Drop unused columns
    columnsToDrop = ['text', 'titleId', 'paragrapghId', 'sentenceId', 'shape', 'isAnswer']
    wordsDf = wordsDf.drop(columnsToDrop, axis = 1)


    return wordsDf

In [6]:
def predictWords(wordsDf, df):
    
    predictorPickleName = 'data/pickles/nb-predictor.pkl'
    predictor = loadPickle(predictorPickleName)
    
    y_pred = predictor.predict_proba(wordsDf)

    labeledAnswers = []
    for i in range(len(y_pred)):
        labeledAnswers.append({'word': df.iloc[i]['text'], 'prob': y_pred[i][0]})
    
    return labeledAnswers

## *Extract questions*

In [7]:
def blankAnswer(firstTokenIndex, lastTokenIndex, sentStart, sentEnd, doc):
    leftPartStart = doc[sentStart].idx
    leftPartEnd = doc[firstTokenIndex].idx
    rightPartStart = doc[lastTokenIndex].idx + len(doc[lastTokenIndex])
    rightPartEnd = doc[sentEnd - 1].idx + len(doc[sentEnd - 1])
    
    question = doc.text[leftPartStart:leftPartEnd] + '_____' + doc.text[rightPartStart:rightPartEnd]
    
    return question


In [8]:
def addQuestions(answers, text):
    doc = nlp(text)
    currAnswerIndex = 0
    qaPair = []

    #Check wheter each token is the next answer
    for sent in doc.sents:
        for token in sent:
            
            #If all the answers have been found, stop looking
            if currAnswerIndex >= len(answers):
                break
            
            #In the case where the answer is consisted of more than one token, check the following tokens as well.
            answerDoc = nlp(answers[currAnswerIndex]['word'])
            answerIsFound = True
            
            for j in range(len(answerDoc)):
                if token.i + j >= len(doc) or doc[token.i + j].text != answerDoc[j].text:
                    answerIsFound = False
           
            #If the current token is corresponding with the answer, add it 
            if answerIsFound:
                question = blankAnswer(token.i, token.i + len(answerDoc) - 1, sent.start, sent.end, doc)
                
                qaPair.append({'question' : question, 'answer': answers[currAnswerIndex]['word'], 'prob': answers[currAnswerIndex]['prob']})
                
                currAnswerIndex += 1
                
    return qaPair

In [9]:
def sortAnswers(qaPairs):
    orderedQaPairs = sorted(qaPairs, key=lambda qaPair: qaPair['prob'])
    
    return orderedQaPairs    

## *Distractors*
Taken from the *04. Generating incorrect answers/Incorrect-answers* notebook.

In [ ]:
import os
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

glove_file = 'data/embeddings/glove.6B.300d.txt'
tmp_file = 'data/embeddings/word2vec-glove.6B.300d.txt'
model = None

if os.path.isfile(glove_file):
    from gensim.scripts.glove2word2vec import glove2word2vec
    glove2word2vec(glove_file, tmp_file)
    model = KeyedVectors.load_word2vec_format(tmp_file)
else:
    print("Glove embeddings not found. Please download and place them in the following path: " + glove_file)

<ipython-input-10-7bcca04c57ee>:12: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, tmp_file)


In [ ]:
def generate_distractors(answer, count):
    answer = str.lower(answer)
    
    ##Extracting closest words for the answer. 
    try:
        closestWords = model.most_similar(positive=[answer], topn=count)
    except:
        #In case the word is not in the vocabulary, or other problem not loading embeddings
        return []

    #Return count many distractors
    distractors = list(map(lambda x: x[0], closestWords))[0:count]
    
    return distractors

In [ ]:
def addDistractors(qaPairs, count):
    if not model:
        print("Glove embeddings not found. Please download and place them in the following path: " + glove_file)
    
    for qaPair in qaPairs:
        distractors = generate_distractors(qaPair['answer'], count)
        qaPair['distractors'] = distractors
    
    return qaPairs

# Main function

In [ ]:
def generateQuestions(text, count):
    
    # Extract words 
    df = generateDf(text)
    wordsDf = prepareDf(df)
    
    # Predict 
    labeledAnswers = predictWords(wordsDf, df)
    
    # Transform questions
    qaPairs = addQuestions(labeledAnswers, text)
    
    # Pick the best questions
    orderedQaPairs = sortAnswers(qaPairs)
    
    # Generate distractors
    questions = addDistractors(orderedQaPairs[:count], 4)
    
    # Print
    for i in range(count):
        display(Markdown('### Question ' + str(i + 1) + ':'))
        print(questions[i]['question'])

        display(Markdown('#### Answer:'))
        print(questions[i]['answer'])
        
        display(Markdown('#### Incorrect answers:'))
        for distractor in questions[i]['distractors']:
            print(distractor)
        
        print()

In [ ]:
text = "Oxygen is a chemical element with symbol O and atomic number 8. It is a member of the chalcogen group on the periodic table, a highly reactive nonmetal, and an oxidizing agent that readily forms oxides with most elements as well as with other compounds. By mass, oxygen is the third-most abundant element in the universe, after hydrogen and helium. At standard temperature and pressure, two atoms of the element bind to form dioxygen, a colorless and odorless diatomic gas with the formula O2. Diatomic oxygen gas constitutes 20.8% of the Earth's atmosphere. As compounds including oxides, the element makes up almost half of the Earth's crust."
naruto = "Naruto was serialized in Shueisha's magazine, Weekly Shōnen Jump from 1999 to 2014, and released in tankōbon (book) form in 72 volumes. The manga was adapted into an anime television series produced by Pierrot and Aniplex, which broadcast 220 episodes in Japan from 2002 to 2007; the English dub of the series aired on Cartoon Network and YTV from 2005 to 2009. Naruto: Shippuden, a sequel to the original series, premiered in Japan in 2007, and ended in 2017, after 500 episodes. The English dub was broadcast on Disney XD from 2009 to 2011, airing the first 98 episodes, and then switched over to Adult Swim's Toonami programming block in January 2014, starting over from the first episode. The English dub is still airing weekly on Adult Swim to this day. Besides the anime series, Pierrot has developed eleven movies and twelve original video animations (OVAs). Other Naruto-related merchandise includes light novels, video games, and trading cards developed by several companies."
ub =  "Competing Conservation Objectives for Predators and Prey: Estimating Killer Whale Prey Requirements for Chinook Salmon Rob Williams 1,2 a , Martin Krkos ek , Erin Ashe 4b , Trevor A. Branch , Steve Clark , Philip S. Hammond Erich Hoyt , Dawn P. Noren , David Rosen 10 , Arliss Winship 11 University of Washington, Seattle, Washington, United States of America, Marine Mammal Research Unit, University of British Columbia, Vancouver, Canada, Department of Zoology, University of Otago, Dunedin, New Zealand, Oceans Initiative, Pearse Island, British Columbia, Canada, School of Aquatic and Fishery Sciences, University of Washington, Seattle, Washington, United States of America, SeaWorld, Orlando, Florida, United States of America, Sea Mammal Research Unit, University of St. Andrews, St. Andrews, Scotland, United Kingdom, Whale and Dolphin Conservation Society, North Berwick, Scotland, United Kingdom, National Marine Fisheries Service, Northwest Fisheries Science Center, Marine Mammal & Seabird Program, National Oceanic and Atmospheric Administration, Seattle, Washington, United States of America, 10 Marine Mammal Research Unit, Fisheries Centre, Department of Zoology, University of British Columbia, Vancouver, British Columbia, Canada, 11 Life Sciences Centre, Department of Biology, Dalhousie University, Halifax, Nova Scotia, Canada Abstract Ecosystem-based management (EBM) of marine resources attempts to conserve interacting species. In contrast to single-species fisheries management, EBM aims to identify and resolve conflicting objectives for different species. Such a conflict may be emerging in the northeastern Pacific for southern resident killer whales ( Orcinus orca ) and their primary prey, Chinook salmon Oncorhynchustshawytscha ). Both species have at-risk conservation status and transboundary (Canada–US) ranges. We modeled individual killer whale prey requirements from feeding and growth records of captive killer whales and morphometric data from historic live-capture fishery and whaling records worldwide. The models, combined with caloric value of salmon, and demographic and diet data for wild killer whales, allow us to predict salmon quantities needed to maintain and recover this killer whale population, which numbered 87 individuals in 2009. Our analyses provide new information on cost of lactation and new parameter estimates for other killer whale populations globally. Prey requirements of southern resident killer whales are difficult to reconcile with fisheries and conservation objectives for Chinook salmon, because the number of fish required is large relative to annual returns and fishery catches. For instance, a U.S. recovery goal (2.3% annual population growth of killer whales over 28 years) implies a 75% increase in energetic requirements. Reducin g salmon fisheries may serve as a temporary mitigation measure to allow time for management actions to improve salmon productivity to take effect. As ecosystem-based fishery management becomes more prevalent, trade-offs between conservation objectives for predators and prey will become increasingly necessary. Our approach offers scenarios to compare relative influence of various sources of uncertainty on the resulting consumption estimates to prioritise future research efforts, and a general ap proach for assessing the extent of conflict between conservation objectives for threatened or protected wildlife where the interaction between affected species can be quantified. Citation: Williams R, Krkos ek M, Ashe E, Branch TA, Clark S, et al. (2011) Competing Conservation Objectives for Predators and Prey: Estimating Killer Whale Prey Requirements for Chinook Salmon. PLoS ONE 6(11): e26738. doi:10.1371/journal.pone.0026738 Editor: Mary O’Connor, University of British Columbia, Canada Received June 13, 2011; Accepted October 2, 2011; Published November 9, 2011 Copyright: 2011 Williams et al. This is an open-access article distributed under the terms of the Creative Commons Attribution License, which permits unrestricted use, distribution, and reproduction in any medium, provided the original author and source are credited. Funding: Funding was provided by the Canada-United States Fulbright program (RW) at University of Washington, The Russell Family Foundation, the British Columbia Marine Planning Fund and SeaWorld-Busch Gardens Conservation Fund, and a Natural Sciences and Engineering Research Council of Canada Post Doctoral Fellowship (MK). The funders had no role in study design, data collection and analysis, decision to publish, or preparation of the manuscrip t. Competing Interests: Steve Clark is an independent contractor to SeaWorld Parks and Entertainment, Inc. (SWPE). He does not have and is not pursuing any patents or products, either in development or marketed, as part of his relationship with SWPE, Inc. Furthermore, there are no relevant competing inte rests that would affect the objective review of this paper. SWPE provided data, but played no role in reviewing the science. The terms of his contract with SWPE, In c. do not alter in any way adherence to policies related to sharing of data and material as described in PLoS ONE policies. The funders had no role in study design, data collection and analysis, decision to publish, or preparation of the manuscript. * E-mail: rmcw@st-andrews.ac.uk a Current address: Sea Mammal Research Unit, University of St. Andrews, Scotland, United Kingdom b Current address: Sea Mammal Research Unit, University of St. Andrews, Scotland, United Kingdom Introduction Ecosystem-based management (EBM) of marine resources is widely recognized as the next step in achieving conservation and fishery objectives, while benefiting from lessons learned from the successes and failures of single-species fisheries management [1–3]. EBM includes incorporation of species interactions when setting conservation and fishery objectives [2,4]. EBM approaches are articulated as part of marine policy in many countries [5–7], although implementation is farther along in some regions than others. For example, predator requirements are formally consid- ered in the management of Antarctic krill fisheries off South Georgia, such that body condition of krill predators can be used as an indicator to trigger reduction in fishery quotas [8]. For North Sea cod ( Gadus morhua ), the International Council for the Exploration of the Seas (ICES) has incorporated estimates of grey seal prey requirements to ensure that this predator is considered when setting fisheries quotas [9]. A related management tool used PLoS ONE | www.plosone.org 1 November 2011 | Volume 6 | Issue 11 | e26738 by ICES is the Ecological Quality Objective, an example of which is that changes in sandeel ( Ammodytes marinus ) fishery management actions are triggered if breeding success of black-legged kittiwakes Rissa tridactyla ) falls below a predetermined threshold [9]. Hence, an EBM approach could involve an explicit allocation of fishery quota for ecosystem or predator needs [10,11]. Oceans policies in Canada and the United States are shifting toward EBM [12,13], but implementation requires that practical choices be made, particularly when there are conflicting objectives for different species [14]. In fact, a key requirement of EBM is identification of conflicting objectives, so that trade-offs can be proposed to achieve multiple objectives [15]. A particularly challenging case occurs when two species are endangered, but one eats the other [16]. Such a situation is emerging for southern resident killer whales (–SRKW’, Orcinus orca ) and Chinook salmon Oncorhynchus tshawytscha ) in the Salish Sea, bordered by northern Washington State (WA), USA and southern British Columbia (BC), Canada. Unlike the mammal-eating transient killer whale population, SRKWs prey exclusively on fish, and specialize almost exclusively on Chinook salmon [17,18]. SRKWs have been assessed as one of the most critically endangered marine mammal populations in US waters [19]. Many of the salmon runs on which this predator depends are also depleted, although not all depleted salmon stocks have been formally listed as endangered [20–22]. In Canada, stocks of commercially valuable marine fish species are normally managed using traditional fisheries-management tools, rather than endangered species legislation [23]. By 1998, the total Canadian catch of all salmon species was at its historic low for the 20 th Century, with Chinook and coho experiencing the most severe declines [24]. Historical (late 1800s to early 1900s) annual run size of Chinook salmon to the Fraser River system was nearly 1 million [25]. However, BC salmon stocks in general are estimated to be at 36% of historical (1800s) run size, and Puget Sound stocks at 8% [26]. A number of scientific and socio-economic factors are considered when deciding whether to list depleted populations of exploited species under endangered species legislation. On the other hand, many species have not been formally evaluated. Consequently, the paucity of exploited species on a country’s endangered species lists does not necessarily indicate favourable conservation status [23]. There is a strong ecological link between killer whales and salmon, with both circumstantial and direct evidence for prey selectivity in resident killer whales [17,18,27]. Critical group size of foraging –northern resident’ killer whales (off northern Vancouver Island) was correlated with inter-annual variability in Chinook salmon abundance, but not with the abundance of four other salmon species in the region [28]. Dedicated field studies have demonstrated prey selectivity, in that resident killer whales were found to target Chinook salmon even when Chinook abundance was low relative to other salmonids [17]. In fact, it has been suggested that resident killer whales are ––highly specialized and dependent on this single salmonid species to an extent that it is a limiting factor in their population dynamics’’ [29]. As a result of this specialization, reduced availability of Chinook salmon is linked to increased adult mortality [29] and reduced reproduction [30] of resident killer whales. One stated U.S. recovery goal for southern resident killer whales is an average annual growth of 2.3% over 28 consecutive years [31], which is expected to increase predation pressure on Chinook salmon stocks. The conservation and management objectives for salmon in this transboundary region are more difficult to sum up in one sentence. The Pacific Salmon Commission is an advisory body formed by the Governments of Canada and the United States to implement the Pacific Salmon Treaty. The Commission aims to offer advice that achieves two broad management objectives, (http://www.psc.org/about_role.htm): ––first, to con- serve the Pacific Salmon in order to achieve optimum production, and second, to divide the harvests so that each country reaps the benefits of its investment in salmon management.’’ Populations of both killer whales and Chinook salmon are in need of rebuilding. However, there is strong potential for these objectives to conflict, given the nearly exclusive dietary speciali- zation on Chinook salmon by the former. Good estimates of prey (energetic) requirements of killer whales are needed to evaluate the extent to which management objectives for these predators and their prey are in conflict, and to offer advice for resolving conflict. Previous attempts to estimate killer whale prey requirements have used metabolic rates measured from two captive animals of unknown mass [32] or estimated field metabolic rates from daily activity budgets of wild whales, scaled up to population-level requirements [33–35]. Drawing inferences from such data is clearly unsatisfactory, but there is little information available about how mass-specific metabolic rates of killer whales depend on sex, reproductive status, age, and size [34]. Conflicts between species occur in many places, but manage- ment actions to mitigate conflicts can have important and potentially undesirable consequences [36]. For example, grey seal culls to protect breeding burrows of puffins on the Farne Islands, UK led to the establishment of new seal colonies and a substantial increase in grey seal population growth [37], increasing conflict with fisheries in the North Sea [38]. More recently, the establishment of Special Areas for Conservation under the EU Habitats Directive for harbour seals and Atlantic salmon in Scotland raises issues similar to those for SRKWs and Chinook salmon [39]. Notwithstanding the difficulties involved in predict- ing ecosystem response to management action, any evaluation of the potential for conflict between fisheries and marine mammals will involve generating good estimates of prey required to sustain and recover the predator population [40]. Using data from several sources, we modelled morphometrics and energetics of killer whales of different age-sex classes. Our analyses draw on extensive records from several killer whale ecotypes and geographic regions based on historic live-capture fisheries and whaling records. The results thereby also provide new parameter estimates that can be used globally in marine ecosystem models that include killer whales. We applied the model to the known sex, size, and age distribution of the 87 individuals in the SRKW population in 2009 (courtesy of Center for Whale Research, Friday Harbor, WA) to estimate the energetic needs and corresponding number of Chinook salmon required for maintenance and recovery of the endangered SRKW population. Finally, we compare the estimates of SRKW consumption of Chinook with fisheries stock assessment data for Chinook salmon from the Fraser River to demonstrate the potential for conflict between objectives for killer whale conservation and salmon fisheries management. Our analyses therefore serve a dual mandate: to provide new parameter estimates for ecosystem models that include killer whales around the world; and to illustrate one way that the approach could be used, using southern resident killer whales as a data-rich case study. Methods We developed a set of statistical models for the relationships among age, length, mass, sex, and energy consumption of captured and captive killer whales and then applied the models to the known age and sex distribution of the SRKW population. Killer Whales and Salmon PLoS ONE | www.plosone.org 2 November 2011 | Volume 6 | Issue 11 | e26738 An important assumption of our approach is that the statistical relationships we estimated for killer whales are transferable among live-capture, captive, and wild populations. We therefore com- pared results from models from these different sources of data as a check, where comparable data were available. The models yielded an estimate of the gross energy require- ments of the SRKW population, which we converted to Chinook salmon consumption based on the average energy content of an individual adult Chinook salmon. The estimated salmon require- ment of the SRKW population was then compared to fisheries data of catch and escapement of Chinook salmon populations upon which the SRKW population feeds. Full details of the modeling approach are given in the supporting information (Text S1). Below we provide a cursory overview. First, we modeled length at age from time-series data of 30 captive killer whales (29 of Icelandic origin and one female northeastern Pacific northern resident ecotype). We fitted and compared three growth models – von Bertalanffy, Gompertz, and logistic growth [41] – using a nonlinear mixed-effects modeling approach to account for the repeated measures structure of the data [42]. The model allowed for differences in growth rates and asymptotic length between male and female whales, thereby capturing the relationships among sex, age, and length. We checked the estimated asymptotic lengths of the models fit to the captive whale data against global capture records from the International Whaling Commission and other sources (Text S1). Next, to check correspondence of allometric relationships between wild and captive animals, we modeled mass-at-length from live-capture records [43] as well as that for captive killer whales held by SeaWorld according to ln( ln( (ln( )) where W is mass in kg and L is length in cm. For live-capture records, we compared models with mixed and fixed effects on ln( and to accommodate possible variation among four ecotypes: Icelandic (n = 11); northern resident (n = 15); southern resident (n = 18); and North Pacific mammal-eating or transient killer whales (n = 4). For the captive killer whales, we included a random effect on the intercept and slope to account for the repeated measures structure of the data [42]. To model gross energy requirements of killer whales, we used data on food consumption from captive animals held by SeaWorld (Text S1; Table S1) according to ln( ln( ln( where is the energy consumed per day in kcal and is the length of the whale in cm. We modeled males and females separately, and further placed females into one of four reproductive categories: (1) –Single’ (neither pregnant, lactating, nor immediately post- lactation); (2) –Pregnant’ (inferred from blood hormone levels); (3) –Lactating’ (recorded until month 3 post-birth); and (4) –Post- lactation’ (six months after lactation records end to allow for possibility of continued partial lactation and a recovery period). The sex and reproductive categories were included as fixed effects in the model and random effects were further included to account for the repeated measures structure of the data [42]. Finally, we applied the above models to the 2009 SRKW population (i.e., 87 individuals) using data on age, sex, and reproductive status as inputs to predict the lengths, weights, and energy needs of SRKWs (demographic data used with permission from Ken Balcomb, Center for Whale Research; see Text S1). Because of the high degree of prey-selectivity of SRKWs on Chinook salmon, we estimated the amount of Chinook salmon required to meet the energetic needs of the SRKW population in 2009. We therefore converted the estimated caloric demand to units of fish, in two ways. First, we considered a value (16,386 kcal/fish) of a Chinook salmon of unknown size sampled in the Salish Sea summer core habitat for SRKWs [34]. Secondly, we placed upper and lower bounds on this measure, by estimating the caloric value of the SRKW’s preferred prey item (a 4-year-old Chinook salmon of length 81 cm and mass 8.5 kg; [17]) using two previously reported estimates of energy density: a ––calorie-rich’’ estimate (18,700 kcal per fish; [44, p. 57]), and a ––lean’’ or energy- sparse estimate (10,869 kcal/fish; [45]). Because our model was based on gross energy consumption from different prey types, our conversion to Chinook biomass assumes a reasonably equivalent transformation from gross to net energy in wild and captive killer w"
generateQuestions(naruto, 4)